In [1]:
import numpy as np 
import pandas as pd

In [2]:
df = pd.read_csv('/home/jho9/Documents/macro_data.csv')

In [3]:
df = df.drop(0)

In [4]:
df.head()

,sasdate,NAPMPI,PAYEMS,T10YFFM,S.P.500,rec_dates
1,1/1/1959,70.7,52480,1.54,55.62,0.0
2,2/1/1959,70.3,52687,1.53,54.77,0.0
3,3/1/1959,71.2,53016,1.19,56.16,0.0
4,4/1/1959,72.1,53320,1.16,57.10,0.0
5,5/1/1959,73.9,53549,1.41,57.96,0.0


In [5]:
#df['sasdate'][:168]
#Start point is 168

In [6]:
features = ['NAPMPI', 'PAYEMS', 'T10YFFM','S.P.500']
target = 'rec_dates'

In [7]:
X = df[features]

In [8]:
#month of is it a recession
#y = df[target]

#next month is it a recession
y = df['rec_dates'].shift(-3)

In [9]:
X = X.iloc[:-3,:]
y = y.iloc[:-3]

In [10]:
from sklearn.naive_bayes import GaussianNB
clf = GaussianNB()

In [11]:
from sklearn.model_selection import TimeSeriesSplit

In [12]:
tkfold = TimeSeriesSplit(n_splits = len(y)-1)

In [13]:
print(tkfold)

TimeSeriesSplit(max_train_size=None, n_splits=684)


In [14]:
X = np.array(X)
y = np.array(y)

In [16]:
#Train up up to 168 then start forcasting in a window after 168
def window_tester(X, y, init_train_indx, clf):
    lst_of_predicted = []
    lst_of_actual = []
    for i in range(init_train_indx + 1,len(y)):
        X_train = X[:i-1,:]
        y_train = y[:i-1]
        X_test = X[[i][:]]
        y_test = y[i]
        clf.fit(X = X_train, y = y_train)
        y_pred = clf.predict(X_test)
        lst_of_predicted.append(y_pred)
        lst_of_actual.append(y_test)
        #print("This is y_pred:   ", y_pred)
        #print("This is y_actual: ", y_test)
    return lst_of_predicted, lst_of_actual

In [17]:
from sklearn.linear_model import LogisticRegression

In [18]:
clf1 = LogisticRegression()

In [19]:
from sklearn.neighbors import KNeighborsClassifier
clf2 = KNeighborsClassifier()

In [20]:
from sklearn.linear_model import Perceptron

In [21]:
clf3 = Perceptron()

In [22]:
from sklearn.svm import LinearSVC

In [23]:
clf4 = LinearSVC()

In [24]:
from sklearn.neural_network import MLPClassifier

In [25]:
clf5 = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(5, 2), random_state=1)


In [58]:
from sklearn.ensemble import RandomForestClassifier

In [60]:
clf6 = RandomForestClassifier()

In [85]:
y_pred, y_true = window_tester(X,y,168, clf)

In [86]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score

In [87]:
confusion_matrix(y_true, y_pred)

array([[423,  21],
       [ 35,  37]])

In [88]:
accuracy_score(y_true, y_pred)

0.8914728682170543

In [89]:
roc_auc_score(y_true, y_pred)

0.7332957957957958

In [90]:
recall_score(y_true, y_pred)

0.5138888888888888

In [91]:
precision_score(y_true, y_pred)

0.6379310344827587

In [92]:
f1_score(y_true, y_pred)

0.5692307692307692

In [93]:
from imblearn.over_sampling import SMOTE

In [94]:
#Train up up to 168 then start forcasting in a window after 168
def window_tester_smote(X, y, init_train_indx, clf):
    sm = SMOTE(random_state=42)
    lst_of_predicted = []
    lst_of_actual = []
    for i in range(init_train_indx + 1,len(y)):
        X_train = X[:i-1,:]
        y_train = y[:i-1]
        X_test = X[[i][:]]
        y_test = y[i]
        X_res, y_res = sm.fit_sample(X_train, y_train)
        clf.fit(X_res, y_res)
        y_pred = clf.predict(X_test)
        lst_of_predicted.append(y_pred)
        lst_of_actual.append(y_test)
        #print("This is y_pred:   ", y_pred)
        #print("This is y_actual: ", y_test)
    return lst_of_predicted, lst_of_actual

In [102]:
y_pred, y_true = window_tester_smote(X,y,168, clf)

In [103]:
confusion_matrix(y_true, y_pred)

array([[406,  38],
       [ 28,  44]])

In [104]:
accuracy_score(y_true, y_pred)

0.872093023255814

In [105]:
roc_auc_score(y_true, y_pred)

0.7627627627627628

In [106]:
recall_score(y_true, y_pred)

0.6111111111111112

In [107]:
precision_score(y_true, y_pred)

0.5365853658536586

In [108]:
f1_score(y_true, y_pred)

0.5714285714285715

In [109]:
from imblearn.under_sampling import RandomUnderSampler

In [110]:
#Train up up to 168 then start forcasting in a window after 168
def window_tester_smote(X, y, init_train_indx, clf):
    sm = RandomUnderSampler(random_state=42)
    lst_of_predicted = []
    lst_of_actual = []
    for i in range(init_train_indx + 1,len(y)):
        X_train = X[:i-1,:]
        y_train = y[:i-1]
        X_test = X[[i][:]]
        y_test = y[i]
        X_res, y_res = sm.fit_sample(X_train, y_train)
        clf.fit(X_res, y_res)
        y_pred = clf.predict(X_test)
        lst_of_predicted.append(y_pred)
        lst_of_actual.append(y_test)
        #print("This is y_pred:   ", y_pred)
        #print("This is y_actual: ", y_test)
    return lst_of_predicted, lst_of_actual

In [111]:
y_pred, y_true = window_tester_smote(X,y,168, clf)

In [112]:
confusion_matrix(y_true, y_pred)

array([[399,  45],
       [ 27,  45]])

In [113]:
accuracy_score(y_true, y_pred)

0.8604651162790697

In [114]:
roc_auc_score(y_true, y_pred)

0.7618243243243243

In [115]:
recall_score(y_true, y_pred)

0.625

In [116]:
precision_score(y_true, y_pred)

0.5

In [117]:
f1_score(y_true, y_pred)

0.5555555555555556

In [118]:
from imblearn.over_sampling import RandomOverSampler

In [119]:
#Train up up to 168 then start forcasting in a window after 168
def window_tester_smote(X, y, init_train_indx, clf):
    sm = RandomOverSampler(random_state=42)
    lst_of_predicted = []
    lst_of_actual = []
    for i in range(init_train_indx + 1,len(y)):
        X_train = X[:i-1,:]
        y_train = y[:i-1]
        X_test = X[[i][:]]
        y_test = y[i]
        X_res, y_res = sm.fit_sample(X_train, y_train)
        clf.fit(X_res, y_res)
        y_pred = clf.predict(X_test)
        lst_of_predicted.append(y_pred)
        lst_of_actual.append(y_test)
        #print("This is y_pred:   ", y_pred)
        #print("This is y_actual: ", y_test)
    return lst_of_predicted, lst_of_actual

In [120]:
y_pred, y_true = window_tester_smote(X,y,168, clf)

In [121]:
confusion_matrix(y_true, y_pred)

array([[396,  48],
       [ 26,  46]])

In [122]:
accuracy_score(y_true, y_pred)

0.8565891472868217

In [123]:
roc_auc_score(y_true, y_pred)

0.7653903903903903

In [124]:
recall_score(y_true, y_pred)

0.6388888888888888

In [125]:
precision_score(y_true, y_pred)

0.48936170212765956

In [126]:
f1_score(y_true, y_pred)

0.5542168674698795

In [249]:
    pos_class_inco = 0
    neg_class_inco = 0
    pos_class_corr = 0
    neg_class_corr = 0
    if y_pred == 1:
            if y_pred == y_test:
                pos_class_corr = pos_class_corr + 1
            else:
                pos_class_inco = pos_class_inco + 1
        elif y_pred == 0:
            if y_pred == y_test:
                neg_class_corr = neg_class_corr + 1
            else:
                neg_class_inco = neg_class_inco + 1
        else:
            print("Error")
    #Overall Accuracy 
    print((neg_class_corr + pos_class_corr) / (neg_class_corr + neg_class_inco + pos_class_corr + pos_class_inco))
    #Precision of Negative
    print(neg_class_corr/(neg_class_corr + neg_class_inco))
    #Recall of Negative
    print(neg_class_corr/(neg_class_corr + pos_class_inco))
    #Precision of Positive
    ppv = pos_class_corr/(pos_class_corr + pos_class_inco)
    print(ppv)
    #Recall of Positive
    tpr = pos_class_corr/(pos_class_corr + neg_class_inco)
    print(tpr)
    #F measure or F1
    print(2 * ((ppv * tpr)/(ppv + tpr)))

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 10)